# Owl
## Retirement planner with great wisdon

This package is a retirement modeling framework for exploring the sensitivity of retirement financial decisions. Strictly speaking, it is not a planning tool, but more an environment for exploring *what if* scenarios. It provides different realizations of a financial strategy. One can certainly have a savings plan, but due to the volatility of financial investments, it is impossible to have a certain asset earnings plan. This does not mean one cannot make decisions. These decisions need to be guided with an understanding of the sensitivity of the parameters.This is exactly where this tool fits it. Given your savings and spending desires, it can generate different future realizations of your strategy under different market assumptions, helping to better understand your financial situation.

The algorithm in Owl is using the open-source HiGHS linear programming solver.
The complete formulation and detailed description of the underlying
mathematical model can be found
[here](https://raw.githubusercontent.com/mdlacasse/Owl/main/docs/owl.pdf).

Copyright &copy; 2024 - Martin-D. Lacasse

Disclaimers: *I am not a financial planner. You make your own decisions. This program comes with no guarantee. Use at your own risk.*

### <span style="color: blue"> README FIRST </span>
<span style="color:black;background:yellow;font-weight:bold">
Don't make changes directly to this file. Keep it as a working example. Therefore, it is recommended that you build your own case by making a copy of the template files provided where you will be able to enter your own numbers and explore your own assumptions.
</span>

# Tutorial - Using the case of Kim and Sam

## Introduction
This file is provided as an example to introduce you to Owl.

It is assumed that you have some familiarity with using a jupyter notebook or jupyterLab, and some basic programming skills in Python. If not, a simple tutorial can guide you to the basic skills needed.

For simulating your own realizations, use the files beginning with *template*. Make a copy and rename them keeping the same extension and give them your own names. Then you'll be able to personalize a case with your own numbers and start experimenting with Owl.

This notebook is an empty template of a ficticious couple Kim and Sam used for demonstration.

### Just some Python module bookkeeping
This command needs to be at the beginning of every Owl notebook.

In [ ]:
# Include if owlplanner not installed as a package.
import sys
sys.path.insert(0, "../src")

In [ ]:
import owlplanner as owl

## Initializing the life parameters for the realization
In order to be able to generate a realization of the future, one must start with providing the year of birth of each spouse(s) and their expected lifespan(s).

For selecting your own numbers, there are plenty of longevity predictors on the web. Pick your favorite:

https://longevityillustrator.org

https://www.livingto100.com/calculator

https://www.sunlife.ca/en/tools-and-resources/tools-and-calculators/life-expectancy-calculator/

or just Google life expectancy calculator.

There are two values needed for couples. Single individuals just enter one value in each list between square brackets `[ ]`. For couples, always keep the same order in the pair of values when entering the data.

When first creating a plan, the default values used for assets allocation ratios and rates of return will be reported. These values are listed to make you aware that these values would be used if no other choice is entered. We will cover how to do this in the next sections.

**Enter your name(s), year(s) of birth, life expectancy(ies), and name for the plan.**

In [ ]:
plan = owl.Plan(['Kim', 'Sam'], [YYYY, YYYY], [AA, AA], 'Kim+Sam-spending', verbose=True)

## Specify account balances
For each spouse, savings accounts have three buckets comprising of the total value of:
- Individual **taxable** investment or savings accounts, including bank accounts, and CDs - do not include your safety net account which should typically be sufficient for sustaining 6 months of living expenses;
- **Tax-deferred** savings accounts, including all IRAs, 401k, 403b, etc.;
- **Tax-free** savings accounts, including Roth IRAs and Roth 401k.

For married couples, each spouse will have to enter values for each type of savings account, following the same order as before. For single individuals, only one value is needed between square brackets `[ ]`.


You can use the `units` optional keyword to indicate that the numbers are entered in other units, such as dollars (`1`) or millions (`M`). Notice that being in Python, arithmetic can be used while entering numbers.

**Enter you account balances for each spouse here.**

In [ ]:
plan.setAccountBalances(
    taxable=[XX, XX],
    taxDeferred=[XX, XX],
    taxFree=[XX, XX],
)

Most investment accounts have named beneficiaries. The *beneficiary* values specify the fraction of total of assets left to the other spouse at death. For example, a spouse leaving 3/4 of her fortune to her three children and the other part to her partner would have a beneficiary value of 0.25.

Benificiary fractions are entered as [1, 1, 1], i.e., one for each type of account. Default is to leave everything to the living spouse.

By default, the account balances are assumed to be known as of today, and amounts will be back projected to Jan 1st using the assumed return rates. An arbitrary date can be chosen as a reference point in the current year, using the optional parameter startDate. For example, using startDate='01-01' would select January 1st, the beginning of the year. This is useful for reproducibility studies. Here is an example:

In [ ]:
plan.setAccountBalances(
    taxable=[XX, XX],
    taxDeferred=[XX, XX],
    taxFree=[XX, XX],
    startDate="01-01"
)

In [ ]:
# plan.setBeneficiaryFractions([1, 1, 1])

## There must be a plan for wages, savings, and Roth conversions
The most manageable part of retirement planning is the control one has over work income, contributions to savings accounts, Roth conversions, and other big spending items in the near- and mid-term future.
In order to execute a realization, one must provide an earning, saving, and Roth conversion plan. This is done through providing an Excel workbook with one spreadsheet (tab) per spouse with the following information:

|year|anticipated wages|taxable ctrb| 401k ctrb | Roth 401k ctrb | IRA ctrb | Roth IRA ctrb | Roth conv | big-ticket items|
|--|--|--|--|--|--|--|--|--|
|2024 | | | | | | | | |
|2025 | | | | | | | | |
| ... | | | | | | | | |
|20XX | | | | | | | | |

Here, 20XX is the last row which could be the last year based on the life expectancy values provided. Missing years will be filled with empty values. For the columns, *anticipated wages* is the annual amount (gross minus tax-deferred contributions) that you anticipate to receive from employment or other sources (not including dividends from your taxable investment accounts). Note that column names are case sensitive and all of these entries must be in lower case. Best way to start this process is to use the template provided rightly named *template.xlsx*.

For the purpose of this exercise, there is no clear definition of retirement age. There will be a year, however, from which you will stop having anticipated income, or diminished income due to decreasing your work load. This transition can be gradual or sudden. Therefore there is no need to enter a retirement age for the sole purpose of quantifying your financial future.

Contributions to your savings accounts are marked as *ctrb*. Contributions to your 401k must also include your employer's contributions, if any. As this file is in Excel, one can use the native calculator to enter a percentage of the anticipated wages for contributions as this can sometimes be easier. Considering a specific example, assume that Jack earns 100k\\$ and contributes 5% to his 401k which his employer matches at up to 4%, then Jack's anticipated wages will be $(1-.05)*100000 = 95000$ and his 401k contributions will be $.09/(1 - .05) * 95000 = 9000 $. 

Roth conversion are specified in the column marked *Roth conv*. Roth conversion are typically performed in the years when the income is lower (and therefore lower tax rates), typically in the bridge years between having a full-time regular salary and collecting social security. This column is provided to override the Roth conversion optimization in Owl. When the solver is given the option `maxConversion='file'`, then these values will be used and no optimization over Roth conversions will be performed. This column is provided for flexibility and allowing comparisons.

Finally, *big-ticket items* are used for accounting for the sale or purchase of a house, or any other major expense or money that you would give or receive (e.g., inheritance, or large gifts to or from you). Therefore, the sign (+/-) of entries in this column is important. Positive numbers will be considered in the cash flow for the year and surplus will be deposited in taxable savings accounts. Negative numbers will generate withdrawals and distributions from retirement accounts. This is the only column that can contain negative numbers: all other column entries should be positive.

The tab name for each spreadsheet represents the name of the spouse for reporting yearly transactions affecting the plan. There has to be one tab for each individual and bearing the same name. Therefore, you need to rename these tabs to have the same names as those used to create the plan (i.e., *Jack* and *Jill* in this case).

Note that the (free) LibreOffice software can be used if you do not have an Excel license, as LibreOffice can read and save `.xlsx` files. Moreover, the native file format from the LibreOffice software can also be read directly. However, you will have to install the `odfpy` package through `conda install conda-forge::odfpy` first.

**Change file name from template to your own Excel worksheet. Copy template.xlsx as a starting example.**

In [ ]:
plan.readContributions('../examples/template.xlsx')

## What are the current and future assets allocations?
Each savings account can invest in 4 major classes of assets:
- Equity funds tracking the S&P 500 index;
- Bond assets tracking the Corporate bonds (Baa) index;
- Fixed-income securities represented by the performance of 10-year Treasury notes;
- Inflation-indexed securities tracking the urban Consumer Price Index (common assets).

The total of percentages in each class of assets for each savings account must add to 100%.

You are asked to provide assets allocation ratios for today, and ones for the time at the end of your life.
Values in between will be interpolated using a linear operator by default. This can be useful
if you want to shift assets allocation as you age. One can also select an *S* curve for transitioning
from the initial value to the final value. This is done by choosing *s-curve* instead of *linear* as the method for interpolation. *S-curve* take two additional parameters, the *center* of the change which defaults to 15 years from now, and the *width* which defaults to 5 years. These parameters will induce
a gradual change from the initial allocations now, starting to change in 10 years (15 - 5), and converging to final allocations in 20 years (15 + 5). 


**Use 'linear' or 's-curve' interpolation.**

**Enter your desired asset allocations over the span of the plan.**

Percentages need to add up to 100%.

In [ ]:
plan.setInterpolationMethod('s-curve')
plan.setAllocationRatios(
    'individual',
    generic=[[[60, 40, 0, 0], [70, 30, 0, 0]], [[60, 40, 0, 0], [70, 30, 0, 0]]]
  )

### Show assets allocations during the time period
The allocation of assets is shown for the three types of savings accounts (taxable, tax-deferred, and tax-free) for each spouse and for the 4 types of investments: stocks (S&P 500), corporate bonds (Baa), Treasury notes (10-y), and common assets tracking inflation.

In [ ]:
plan.showAllocations()

## What about anticipated fixed income?
Pension and social security are fixed income. Model here assumes that pension income is not inflation adjusted while social security benefits are (but Owl can easily be modified to account for inflation-adjusted pensions). Numbers to be provided are the predicted annual amount for each spouse and the age of the commencement of benefits.

By default, no pension benefits are assumed. This can also be specified explicitly by entering zeros (0) as entries, as in

    plan.setPension([0, 0], [65, 65])
    
For social security, one must provide the predicted annual amount(s) and the starting age(s) at which benefits are anticipated to be received. There are plenty of social security benefit estimators on the web, including the info you can get directly from your own account at the Social Security Administration (ssa.gov). Another interesting calculator can be found at www.opensocialsecurity.com. This calculator allows you to compare different scenarios regarding your commencement age through a sensitivity plot.


**Enter your anticipated social security benefits, and age of benefits.**

In [ ]:
plan.setPension([XX, XX], [AA, AA])
plan.setSocialSecurity([XX, XX], [AA, AA])

## How much net income is desirable at retirement?
For determining the desirable annual net income in retirement, certified planners will strongly suggest that you've must have already done a cash flow analysis on your yearly spending. After this exercise, you should have a good idea of how much you'll need in retirement. Another approach is to experiment with multiple spending scenarios and see what your current and future savings can sustain under different market conditions. As one nears retirement, both approaches will need to meet at one point.

The desired income defined here is the minimum annual **net** income (i.e., after paying federal income tax) that one would like to receive starting at her/his "retirement age" (we provided a loose definition of the term *retirement age* above). This desired income must be adjusted for inflation and can follow an additional adjustment called a *smile* profile. A *smile* profile accounts for the fact that your spending capacity will modulate during retirement as you go from the so-called gogo years to the no-go years. A *flat* profile, on the other hand, will keep the same value, which will only be adjusted for inflation. More realistically, the income could also be modulated by the performance of assets, reducing withdrawals in market down years. These strategies (first proposed by Guyton and Klinger) could be implemented in Owl. 

A second optional argument can be provided to specify the fraction of income left the the survivor. The default is 60%.

The target and actual net income values achieved through the realization can be plotted as will see below.


**Use a *smile* curve or a *flat* line.**

In [ ]:
plan.setSpendingProfile('smile')
plan.showProfile()

## Specify rates of return and inflation rate
Rates of return for each class of assets can be specified, including the rate of inflation.
These are important components for  future assumptions.
Setting future rates is done with the `setRates()` method from which one can select different
sources for determining the rates.
Valid choices are *historical*, *histochastic*, *historical average*, *fixed*, *conservative*, *stochastic*, or *default*.

For the *optimistic* case, values reported from MorningStar for the next ten years are used. It is selected by using this call:

    plan.setRates('optimistic')

*Conservative* rates are lower than the last ten years as most analysts are predicting lower rates from the next decade. These values are fixed rates of 6.0% for the S&P 500, 4.0% for the Corporate bonds markets, 3.5% for the 10-year Treasury notes, with an inflation at around 2.8%. Using these conservative rates can be achieved by using the following call:

    plan.setRates('conservative')

For the *historical*, *stochastic*, and *average* options, data from 1928 to the last year are available for experimenting.
Ranges chosen smaller than the life horizon of the longest-lived individual will have rate values repeated in cycle. For example,
choosing historical data from 1994 to (up and including) 1996 will repeat these three values over the time span of the realization.
This would be called as follows:

    plan.setRates('historical', 1994, 1996)

This case is only provided as an explanatory example, as it would have little practical value.

If the upper bound is not provided as the third argument, then the latest data year (i.e., last year) will be assumed by default.
If one chooses a historical range starting from 1970, Owl will use the rates of 1970 for this year and 1971 for next, etc. This would be simulated as follows:

    plan.setRates('historical', 1970)
  
Due to its particular sequence of rates, the worst-case historical scenario is a retirement starting in 1966. This can be simulated as follows:

    plan.setRates('historical', 1966)

In this case, the  current year will have the same rates as those that happened in 1966, and next year will have those from 1967, and so on. This choice is given for instructional purposes only. No one should make a plan based on the worst-case historical scenario. Nevertheless, it can be informative to test your own case. In practice, however, a success rate larger than 90% over a reasonable set of historical starting years and market assumptions would be acceptable by a large portion of rational thinkers. But this is all a personal choice. We'll cover how to model multiple starting years below.

Alternatively, one can choose a *histochastic* approach in which case the rates are determined from the multivariate distribution for the 4 rates in the selected year range.  The computed statistical distribution of the selected range of data is used to generate random rate values. For example,

    plan.setRates('histochastic', 1945)
    
will analyze the annual rates from 1945 up to last-year and compute means and covariance to generate new data that are statistically representative of the ones observed during this selected time period. The rates randomly generated for the time span can be plotted and examined as we will see below. Similarly,

    plan.setRates('histochastic', 1940, 1970)

would generate random rates consistent to those observed during the 1940 - 1970 time period.

Rates can also be generated stochastically by choosing the *stochastic* option.
In its simplest case, mean values, and standard deviations (volatility) need to be provided (in percent) as follows:

    my_means = [8, 5, 4, 3]
    my_stdev = [17, 8, 8, 2]
    p.setRates('stochastic', values=my_means, stdev=my_stdev)
    
Here, we select 8% return on S&P 500 with 17% volatility, 5% return on corporate bonds and and 4% on T-notes, both with 8% volatility, and 3% inflation with 2% of volatility.

Called as shown will assume that there is no correlation between rates of return (in mathematical terms,
the correlation matrix defaults to the identity matrix). However, we know that rates are somehow correlated (i.e., when the inflation goes up, returns on bonds tend to go down). To account for this coupling, we
can either provide a correlation matrix as here:
    
    my_means = [8, 5, 4, 3]
    my_stdev = [17, 8, 8, 2]
    my_corr = [[1, 0.46, 0.06, -.12], [0.46, 1, 0.68, -.27], [0.06, 0.68, 1, -.21], [-.12, -.27, -.21, 1]]
    p.setRates('stochastic', values=my_means, stdev=my_stdev, corr=offdiag_corr)

or, as the correlation matrix is symmetric, just provide the 6 off-diagonal elements as shown here:

    my_means = [8, 5, 4, 3]
    my_stdev = [17, 8, 8, 2]
    offdiag_corr = [.46, .06, -.12, .68, -.27, -.21]
    p.setRates('stochastic', values=my_means, stdev=my_stdev, corr=offdiag_corr)

In standard matrix notation, the off-diagonal elements of correlation matrix $C$ are $c_{12}, c_{13}, c_{14}, c_{23}, c_{24}, c_{34}$. As $c_{ii} = 1$ and $C$ is symmetric, it can easily be reconstructed by Owl.

To get an idea of what to expect as correlations, one can use the `getRatesDistributions(from, to)` over a range of
years of historical data as follows:

    means, stdev, corr, covar = owl.getRatesDistributions(1970, 2019)

This call will analyze the data over the year range and report back on the statistical correlations found.

Rates can also be set to fixed values obtained from an average over a time interval using the *average* option. For example, the call

    plan.setRates('average', 1990, 2020)

would set the rates to constant values being the average observed from 1990 to 2020 inclusively. The word 'means' can also be used.
    
Finally one can also use fixed annual rates by providing a list of 4 entries in percent as follows:

    myrates = [9.6, 4.0, 3.0, 3.8]
    plan.setRates('fixed', values=myrates)
    
This example would use fixed rates of 9.6%, 4.0%, 3.0%, and 3.8% as average annual returns on S&P 500, corporate bonds, Treasury notes, and common assets, respectively, with an average annual inflation rate of 3.8% for the full duration of the time simulation. Recall that the common assets class consists of investments tracking inflation only. Therefore the last index serves both to track the common asset class and to adjust values for inflation.

Also note that the S&P 500 rates provided always include dividends, which are assumed to be reinvested.


**Pick the rates you would like to use.**

In [ ]:
plan.setRates('historical', 1969)

#### Show annual rates used for calculations
As described above, there are many choices for selecting rates. This graph will display the annual rates used during the time span of this realization. Numbers between <> are the average over the period considered.

In [ ]:
plan.showRates()

##### Show historical rate distribution
Since Owl has the historical rates available, one can also display their histograms using a simple function call. This is done with `owl.showRatesDistributions()`.

Given the standard deviation of each histogram, the risk/benefit between stocks and bonds is clear. Let's look at the rates distribution over the 30-year period running from 1969 to 1999. Notice how high is the average inflation.

In [ ]:
means, stdev, corr, covar = owl.getRatesDistributions(1969, 1999)
owl.showRatesDistributions(1969, 1999)

When *stochastic* or *histochastic* rates are used, the distributions and correlations betwen the different rates can be displayed by using the following method, (which can optionally share the same percentage range across all investments): 

In [ ]:
# plan.showRatesCorrelations(shareRange=False)

### Other configurable parameters
There are a few more parameters that can be configured. The following calls are provided as examples. If not specified, the default values are:
- Heirs marginal tax rate: 30%
- Tax rate on long-term capital gains: 15%
- Dividend return rate on equities: 2%
- Ability to have graphs plots by default in *nominal* values or *today*'s dollars.
  
These function calls are provided for reference with their default values.

In [ ]:
# plan.setHeirsTaxRate(30)
# plan.setLongTermCapitalTaxRate(15)
# plan.setDividendRate(2)
# plan.setDefaultPlots('nominal')

## Generating the outcome of a scenario
We're now ready to run a single instance of a scenario. 

### Maximum net income
The following function call runs all the required calculations for the time horizon over your life expectancy. The function `solve()` will optimize the selected scenario using linear programming. But we first need to select what needs to be optimized by passing options. If one individual, say 'Jane' cannot perform Roth conversions, it can be specified as:
```python
'noRothConversions': 'Jane'
```
If no Roth conversion is desired for both individuals:
```python
'maxRothConversion': 0
```

**Pick your objective function and related parameters.**

In [ ]:
options = {'maxRothConversion': XX, 'bequest': XX}
plan.solve('maxSpending', options=options)

### Analysis

#### Show net spending compared to target income over the years
This graph shows how the actual net income generated by the plan realization matches the inflation-adjusted net income profile specified. Note the 40% drop in target income as one spouse passes. This drop is configurable through the `setProfile()` method. Default value is 60%.

In [ ]:
plan.showNetSpending()

Each plot can be called with the `value` parameter set to *nominal* or *today*, resulting in a graph that accounts for inflation or not.

In [ ]:
plan.showNetIncome(value='today')

#### Show taxable ordinary annual income and anticipated tax brackets
Gross income also includes Roth conversions and big-ticket items, both of which are not contributing to your net income. This graph shows Jack and Jill's gross taxable income and how it compare with some anticipated Federal tax marginal brackets. This visualization is very convenient when one wants to perform Roth conversions and remain below a certain tax bracket.

Note the shift in tax brackets taking place as the Tax Cut and Job Act expires after 2025.

In [ ]:
plan.showGrossIncome()

#### Show annual taxes paid over the years of the realization
This graph shows how much Jack and Jill paid in federal taxes over the years of this realization.

In [ ]:
plan.showTaxes()

#### Show sources of income over the years
Income will typically come from multiple sources, and it can be quite complex. This graphs shows the breakdown of Jack and Jill's sources of income by spouse and by origin. Note that distributions from tax-deferred accounts (*dist*) are distinguished from required minimum distributions (*rmd*) as they serve different purposes. Other labels should be self-explanatory.

In [ ]:
plan.showSources()

#### Show savings accounts balances at the beginning of each year
The balance for each savings account for each spouse is calculated at the beginning of each year. Another important aspect of this graph is how much is left at the end of the realization. This will be addressed in the next cell.

In [ ]:
plan.showAccounts()

To get more information about accounts at the last year of the realization, one can use

      plan.summary()
    
which returns the value of the assets in today's \\$ at the last year of the scenario, assuming (in this case) a 30\% tax burden on the taxable portion of the estate (read tax-deferred savings accounts). The `summary()` function returns two values: the total post-tax value of all savings account in today's dollars and the cumulative inflation rate between today and the last day of the realization.

In [ ]:
plan.summary()

#### Can we save this realization?
This instance of a future realization contains information on the distribution amounts, including the required minimum distribution that had to be performed under the given assumptions. This info can be saved in an excel workbook with one spreadsheet (tab) for each spouse. Worksheet will also contain annual rates, income, income taxes, and account balances. The argument overwrite as `True/False` controls if existing file, if any, gets overwritten. Also remember that Windows will not allow the file to be overwritten while the file is being opened in Excel. In that case, the script will ask you to close the file and will retry to save.

This call will save the results from the last optimization, i.e., maxSpending.

Open the file in Excel to see what it looks like.

In [ ]:
plan.saveWorkbook(overwrite=True)

## Maximum bequest
Alternatively, one could optimize for maximum bequest subject to a net income constraint.

**Pick your parameters.**

In [ ]:
plan.rename('Kim+Sam-bequest')
options = {'maxRothConversion': XX, 'netSpending': XX}
plan.solve('maxBequest', options=options)

#### Analysis

In [ ]:
plan.showNetSpending()

In [ ]:
plan.showGrossIncome()

In [ ]:
plan.showTaxes()

In [ ]:
plan.showSources()

In [ ]:
plan.showAccounts()

In [ ]:
plan.summary()

### Can we save this realization?
This instance of a future realization contains information on the distribution amounts, including the required minimum distribution that had to be performed under the given assumptions. This info can be saved in an excel workbook with one spreadsheet (tab) for each spouse. Worksheet will also contain annual rates, income, income taxes, and account balances. Second `True/False` argument controls if existing files get overwritten. Also remember that Windows will not allow the file to be overwritten while the file is being opened in Excel. In that case, the script will ask you to close the file and will retry to save.

This call will save the results from the last optimization, i.e., maxBequest.

In [ ]:
plan.saveWorkbook(True)

This notebook is provided as a basic example on what you can do to assess the robustness and sensitivity of your retirement strategy. Go and explore!


Enjoy!
